In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import urllib.request # make request for get smiles of compounds from CFM

from rdkit import Chem, RDLogger, DataStructs 
from rdkit.Chem import Descriptors, AllChem, MACCSkeys, Draw
from rdkit.Chem.rdmolfiles import SmilesMolSupplier, ForwardSDMolSupplier
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import Image

from validation import modernization, filter, number_sign_in_protocol

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [2]:
class Molecule:
    """
    class Molecule represents molecules
    
    Attributes
    ----------
    cid: PubChem Compound ID
    name : name of molecule
    smiles: SMILES
    database: database from which the molecule is taken
    mol : кeading molecule from SMILES

    Methods
    -------
    get_RDKFingerprint()
        return topological fingerprints of molecule
    get_MACCSKeys()
        return MACCS Keys fingerprints of molecule
    getMorganFingerprint()
        return Morgan fingerprints of molecule
    """
    
    def __init__(self, cid, name, smiles, database):
        self.cid = cid
        self.name = name
        self.smiles = smiles
        self.database = database
        self.mol = Chem.MolFromSmiles(self.smiles)

        
    def get_RDKFingerprint(self):
        return Chem.RDKFingerprint(self.mol)
    
    def get_MACCSKeys(self):
        return MACCSkeys.GenMACCSKeys(self.mol)
    
    def getMorganFingerprint(self):
        return AllChem.GetMorganFingerprintAsBitVect(self.mol, 2, nBits = 1024)

In [3]:
class Pair:
    """
    class Pair represents a pair of molecules
    
    Attributes
    ----------
    first_compound: the first molecule from the pair
    second_compound : the second molecule from the pair

    Methods
    -------
    get_simmilarity_using_RDKFingerprint()
        return Tanimoto similarity of molecules using topological fingerprints
    get_simmilarity_using_MACCSKeys_Fingerprint()
        return return Tanimoto similarity of molecules using MACCS Keys fingerprints
    get_simmilarity_using_Morgan_Fingerprint()
        return return Tanimoto similarity of molecules using Morgan fingerprints
    """
    def __init__(self, first_compound, second_compound):
        self.first_compound = first_compound
        self.second_compound = second_compound
        
    def get_simmilarity_using_RDKFingerprint(self):
        RDKFingerprint_first_compound = self.first_compound.get_RDKFingerprint()
        RDKFingerprint_second_compound = self.second_compound.get_RDKFingerprint()
        return DataStructs.FingerprintSimilarity(RDKFingerprint_first_compound, RDKFingerprint_second_compound)
    
    def get_simmilarity_using_MACCSKeys_Fingerprint(self):
        MACCSKeyFingerprint_first_compound = self.first_compound.get_MACCSKeys()
        MACCSKeyFingerprint_second_compound = self.second_compound.get_MACCSKeys()
        return DataStructs.FingerprintSimilarity(MACCSKeyFingerprint_first_compound, MACCSKeyFingerprint_second_compound)
    
    def get_simmilarity_using_Morgan_Fingerprint(self):
        MorganFingerprint_first_compound = self.first_compound.getMorganFingerprint()
        MorganFingerprint_second_compound = self.second_compound.getMorganFingerprint()
        return DataStructs.FingerprintSimilarity(MorganFingerprint_first_compound, MorganFingerprint_second_compound) 

In [4]:
def get_list_pairs_moleculs(list_molecules_1, list_molecules_2):
    """
    make pairs of molecules
    
    Parametrs
    ---------
    list_molecules_1: list with molecules from first database
    list_molecules_2: list with molecules from second database
    
    Return
    ------
    list with pairs of molecules (molecule from list_molecules_1, molecule from list_molecules_2), in that each pair belongs to the class Pair
    """
    
    list_of_pairs = []
    for i in range(len(list_molecules_1)):
        for j in range(len(list_molecules_2)):
            list_of_pairs.append(Pair(list_molecules_1[i], list_molecules_2[j]))
    return list_of_pairs

In [17]:
def select_pairs_by_treshold(list_of_pairs, type_fingerprints, treshold = 0.7):
    """
    select pairs of molecules whose similarity coefficient is greater than the threshold
    
    Parametrs
    ---------
    list_of_pairs: list with pairs of molecules
    type_fingerprints: type of molecular fingerprints
    treshold: set threshold, the default value is 0.7
    
    Return
    ------
    list with selected pairs of molecules, in that each pair belongs to the class Pair
    """
    
    selected_pairs = []
    if type_fingerprints== 'RDKFingerprint':
        for pair in list_of_pairs:
            if pair.get_simmilarity_using_RDKFingerprint() >= treshold:
                selected_pairs.append(pair)
        return selected_pairs
                
    elif type_fingerprints == "MACCSkeys":
        for pair in list_of_pairs:
            if pair.get_simmilarity_using_MACCSKeys_Fingerprint() >= treshold:
                selected_pairs.append(pair)
        return selected_pairs
                
    elif type_fingerprints == "MorganFingerprint":
        for pair in list_of_pairs:
            if pair.get_simmilarity_using_Morgan_Fingerprint() >= treshold:
                selected_pairs.append(pair)
        return selected_pairs

In [7]:
L1000FWD_drug = pd.read_table('DATA/Drugs_metadata.csv', sep = ',')
print('shape of dataframe L1000FWD_drug:', L1000FWD_drug.shape)
L1000FWD_drug.head()
L1000FWD_drug = L1000FWD_drug.dropna(subset = ['canonical_smiles', 'pert_iname', 'pubchem_cid'])

shape of dataframe L1000FWD_drug: (20449, 13)


In [8]:
molecules_L1000FWD_drug = []
for (smiles, name, cid) in zip(L1000FWD_drug['canonical_smiles'], L1000FWD_drug['pert_iname'], L1000FWD_drug['pubchem_cid']):
    molecule = Molecule(cid, name, smiles, 'L1000FWD') # представляем молекулы в виде объектов класса Molecule
    molecules_L1000FWD_drug.append(molecule)

In [9]:
len(molecules_L1000FWD_drug)

20363

In [10]:
chemicals_CFM = pd.read_table('DATA/direct_reprogramming_non-genetics - Chemicals.csv', sep = ',')
print('shape of dataframe chemicals_CFM:', chemicals_CFM.shape)
chemicals_CFM = chemicals_CFM.dropna(subset = ['cid', 'name'])
chemicals_CFM.head()

shape of dataframe chemicals_CFM: (158, 5)


,cid,link,name,Synonyms,MOA
0,459803,https://pubchem.ncbi.nlm.nih.gov/compound/459803,AC1LA18U,NaN,Inhibitor of the HIF prolyl 4-hydroxylase
1,91899426,https://pubchem.ncbi.nlm.nih.gov/compound/9189...,2-phospho-L-ascorbic acid,2-phospho-L-ascorbic acid;BDBM92477,_
2,286003,https://pubchem.ncbi.nlm.nih.gov/compound/286003,DTXSID80301486,NaN,Agonist of the Adenosine Receptor
3,47289,https://pubchem.ncbi.nlm.nih.gov/compound/47289,4-(Methylnitrosamino)-1-(3-pyridyl)-1-butanone,NaN,_
4,451668,https://pubchem.ncbi.nlm.nih.gov/compound/451668,5-Aza-2'-deoxycytidine,Decitabine;5-Aza-2'-deoxycytidine;2353-33-5;Da...,Inhibitor of DNA methylation


In [11]:
def get_smiles_for_molecules_CFM(chemicals_CFM):
    """
    get smiles of molecules by cid of molecules and wtite smiles in file "smiles_of_molecules_CFM.txt"
    
    Parametrs
    ---------
    chemicals_CFM: dataframe with column with cids of molecules
    """
        
    str_with_cids = str()
    for cid in chemicals_CFM['cid']:
        if cid != '_':
            url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + str(cid) + '/property/CanonicalSMILES/csv'
            response = urllib.request.urlopen(url)
            str_response = response.read().decode('utf-8')
            list_responce = str_response.split(',')
            cid_with_extra_quotes = list_responce[-1]
            str_with_cids = str_with_cids + cid_with_extra_quotes
    with open("DATA/intesect_CFM_L1000FWD/smiles_of_molecules_CFM.txt", "w") as file:
        file.write(str_with_cids)

In [ ]:
get_smiles_for_molecules_CFM(chemicals_CFM) # записали smiles в файл

In [12]:
def list_molecules_chemicals_CFM(chemicals_CFM, file_with_smiles):
    """
    make list of molecules, in which each molecule belongs class Molecule
    
    Parametrs
    ---------
    chemicals_CFM: dataframe with column with cids, names of molecules
    file_with_smiles: file with smiles of molecules
    
    Return
    ------
    list of molecules, in which each molecule belongs class Molecule
    """
    
    list_smiles_of_molecules_CFM = []
    with open(file_with_smiles, "r") as file:
        for line in file:
            list_smiles_of_molecules_CFM.append(line[1:-2])
        
    molecules_chemicals_CFM = []
    for (name, cid, smiles) in zip(chemicals_CFM['name'], chemicals_CFM['cid'], list_smiles_of_molecules_CFM):
        molecule = Molecule(cid, name, smiles, 'CFM')
        molecules_chemicals_CFM.append(molecule)
    return molecules_chemicals_CFM

In [13]:
molecules_chemicals_CFM = list_molecules_chemicals_CFM(chemicals_CFM, 'DATA/smiles_of_molecules_CFM.txt')

In [14]:
list_pairs = get_list_pairs_moleculs(molecules_L1000FWD_drug, molecules_chemicals_CFM)
len(list_pairs)

3135902

Отберем пары молекул, коэффициент сходства которых равен 1 и используются Morgan fingerprints.

In [18]:
selected_pairs_1 = select_pairs_by_treshold(list_pairs, "MorganFingerprint", treshold = 1)
print('число пар :', len(selected_pairs_1))

число пар : 104


In [19]:
selected_pairs_07 = select_pairs_by_treshold(list_pairs, "MorganFingerprint", treshold = 0.7)
print('число пар :', len(selected_pairs_07))

число пар : 164


Посмотрим таблицу для клеточных переходов.

In [21]:
structure_CFM = pd.read_table('DATA/direct_reprogramming_non-genetics - structure.csv', sep = ',')
print('shape :', structure_CFM.shape)
structure_CFM.head()

shape : (169, 19)


,DOI,Species,Source Cell Type,Target Cell Type,"Duration, d","name of chemical 1,CID 1;name of chemical 2,CID 2",Medium,Growth Factors,Initial Culture (text),MoA chem 1; MoA chem 2,TFs/miRNA,Title,Assocoated Data,Yield (%),Comment on Yield,Type,Comment,"Authors;sorted=""Reverse"";sep=;",Stress factors
0,10.1371/journal.pone.0089678,Mus musculus,Fibroblasts,Induced Cardiomyocytes,14,"SB431542,4521392",doxycycline (2 µg mL−1) and the PGK-H2B-m Cher...,_,"MEFs, isolated at E14.5",Inhibitor of the Activin/BMP/TGF-β pathway,_,Inhibition of TGFβ signaling increases direct ...,GSE54022,16.95,NaN,2,_,John D. Gearhart; Jonathan A. Epstein; Russell...,_
1,10.1007/s12015-013-9477-9,Sus scrofa,Fibroblasts,Induced Pancreatic Beta Cells,36,"5-Aza-2'-deoxycytidine,9444","N2B27 with 0.1 mM β- mercaptoetanol (Sigma), 2...",basic fibroblast growth factor 20 ng/ml,Primary porcine skin fibroblast cultures were ...,Inhibitor of DNA methylation,_,Reprogramming of pig dermal fibroblast into in...,_,38.1,Counting insulin labeled cells with a flow cyt...,1,_,F.; Gandolfi; T. A. L.; Brevini; M. M.; Rahman...,_
2,10.1038/ncomms3183,Homo sapiens,Fibroblasts,Induced Cholinergic Neurons,28,"Forskolin,47936;Dorsomorphin,11524144",_,FGF2 (10 ng ml−1),Human fibroblasts were obtained from commercia...,"CAMP agonist;Inhibitor of ALK2, ALK3, ALK6",NGN2,Small Molecules Enable Neurogenin 2 to Efficie...,GSE45954,57.2,NaN,2,_,Oliver Brüstle; Philipp Koch; Franz-Josef Müll...,_
3,10.1016/j.biocel.2013.04.022,Homo sapiens,Mesenchymal Stem Cells (Bone marrow derived),Induced Neurons,28,"SB431542,4521392;Dorsomorphin,11524144",NeuroCult supplemented with 1% penicillin/stre...,10 ng/ml recombinant human bFGF,8-week fetal forebrain,Inhibitor of the Activin/BMP/TGF-β pathway;Inh...,_,Enhancing the efficiency of direct reprogrammi...,_,1,"Sodium current, Electrophysiological recordings",1,_,Zhiying Zhang; Qing-song Liu;Alexanian Arshak R.,_
4,10.1371/journal.pone.0003531,Mus musculus,Primordial Germ Cells,Induced Pluripotent Epiblast Cells,10,"Trichostatin A,444732","LIF (1200 IU/ml), FGF-2 (25 ng/ml)",FGF-2 (25 ng/ml),"EG cell lines, 8.5 EGC-1 and 4-3 Rosa, were de...",Inhibitor of the histone deacetylases I and II,_,Reprogramming Primordial Germ Cells into Pluri...,_,30,cells were stained for TNAP activity and the n...,2,_,M. Azim Surani; Reuben Tooze; Gina Doody; Fuch...,_


На основе этой таблицы создадим таблицу, в которой к каждому клеточному переходу укажем пары молекул (первая молекула принадлежит базе L1000FWD, вторая молекула - CFM), которые могут осуществить этот переход, исходя из того, что в таблице structure_CFM указаны cids молекул из базы CFM, которые обладают потенциал для перепрограммирования. То есть, если вторая молекула из пары указана для клеточного перехода, то считаем что первая молекула из пары тоже может осуществить этот переход.

In [25]:
def add_mol_L1000FWD(selected_pairs, structure_CFM):    
    dataframe_selected_pairs_with_features = structure_CFM.iloc[:, 0:6]
    dataframe_selected_pairs_with_features['smiles of chemicals'] = 'not molecules'
    dataframe_selected_pairs_with_features['name of chemicals in L1000FWD'] = str()
    dataframe_selected_pairs_with_features['cid of chemicals in L1000FWD'] = str()
    for pair in selected_pairs:
        for i in range(structure_CFM.shape[0]):
            string_molecules = structure_CFM.iloc[i, 5]
            list_molecules = string_molecules.split(';')
            list_cids = []
            for mol in list_molecules:
                list_cids.append(mol.split(',')[-1])
            if pair.second_compound.cid in list_cids:
                if dataframe_selected_pairs_with_features.iloc[i, 6] == 'not molecules':
                    dataframe_selected_pairs_with_features.iloc[i, 6] = pair.first_compound.smiles + ';'+\
                    pair.second_compound.smiles
                else :
                    dataframe_selected_pairs_with_features.iloc[i, 6] = \
                    dataframe_selected_pairs_with_features.iloc[i, 6] + ';' + pair.first_compound.smiles + ';' +\
                    pair.second_compound.smiles
                    
                
                dataframe_selected_pairs_with_features.iloc[i, 7] = \
                    dataframe_selected_pairs_with_features.iloc[i, 7]  + pair.first_compound.name + ';'
                dataframe_selected_pairs_with_features.iloc[i, 8] = \
                    dataframe_selected_pairs_with_features.iloc[i, 8]  + pair.first_compound.cid + ';'
    return dataframe_selected_pairs_with_features
                    

In [29]:
data_intersect_CFM_L1000FWD_1 = add_mol_L1000FWD(selected_pairs_1, structure_CFM)
print(data_intersect_CFM_L1000FWD_1.shape)
print(data_intersect_CFM_L1000FWD_1[data_intersect_CFM_L1000FWD_1['cid of chemicals in L1000FWD'] != str()].shape)
data_intersect_CFM_L1000FWD_1.head()

(169, 9)
(133, 9)


,DOI,Species,Source Cell Type,Target Cell Type,"Duration, d","name of chemical 1,CID 1;name of chemical 2,CID 2",smiles of chemicals,name of chemicals in L1000FWD,cid of chemicals in L1000FWD
0,10.1371/journal.pone.0089678,Mus musculus,Fibroblasts,Induced Cardiomyocytes,14,"SB431542,4521392",ONC(=O)CCCCCN1C(=O)c2cccc3cccc(C1=O)c23;C1=CC2...,scriptaid;,5186;
1,10.1007/s12015-013-9477-9,Sus scrofa,Fibroblasts,Induced Pancreatic Beta Cells,36,"5-Aza-2'-deoxycytidine,9444",not molecules,,
2,10.1038/ncomms3183,Homo sapiens,Fibroblasts,Induced Cholinergic Neurons,28,"Forskolin,47936;Dorsomorphin,11524144",CC(=O)O[C@H]1[C@@H](O)C2C(C)(C)CC[C@H](O)[C@]2...,colforsin;forskolin;forskolin;forskolin;dorsom...,5702265;47936;3413;6610344;11524144;
3,10.1016/j.biocel.2013.04.022,Homo sapiens,Mesenchymal Stem Cells (Bone marrow derived),Induced Neurons,28,"SB431542,4521392;Dorsomorphin,11524144",ONC(=O)CCCCCN1C(=O)c2cccc3cccc(C1=O)c23;C1=CC2...,scriptaid;dorsomorphin;,5186;11524144;
4,10.1371/journal.pone.0003531,Mus musculus,Primordial Germ Cells,Induced Pluripotent Epiblast Cells,10,"Trichostatin A,444732",not molecules,,


In [28]:
data_intersect_CFM_L1000FWD_07 = add_mol_L1000FWD(selected_pairs_07, structure_CFM)
print(data_intersect_CFM_L1000FWD_07.shape)
print(data_intersect_CFM_L1000FWD_07[data_intersect_CFM_L1000FWD_07['cid of chemicals in L1000FWD'] != str()].shape)
data_intersect_CFM_L1000FWD_07.head()

(169, 9)
(148, 9)


,DOI,Species,Source Cell Type,Target Cell Type,"Duration, d","name of chemical 1,CID 1;name of chemical 2,CID 2",smiles of chemicals,name of chemicals in L1000FWD,cid of chemicals in L1000FWD
0,10.1371/journal.pone.0089678,Mus musculus,Fibroblasts,Induced Cardiomyocytes,14,"SB431542,4521392",ONC(=O)CCCCCN1C(=O)c2cccc3cccc(C1=O)c23;C1=CC2...,scriptaid;itdac-1;,5186;3785;
1,10.1007/s12015-013-9477-9,Sus scrofa,Fibroblasts,Induced Pancreatic Beta Cells,36,"5-Aza-2'-deoxycytidine,9444",not molecules,,
2,10.1038/ncomms3183,Homo sapiens,Fibroblasts,Induced Cholinergic Neurons,28,"Forskolin,47936;Dorsomorphin,11524144",CC(=O)O[C@H]1[C@@H](O)C2C(C)(C)CC[C@H](O)[C@]2...,colforsin;forskolin;forskolin;forskolin;dorsom...,5702265;47936;3413;6610344;11524144;
3,10.1016/j.biocel.2013.04.022,Homo sapiens,Mesenchymal Stem Cells (Bone marrow derived),Induced Neurons,28,"SB431542,4521392;Dorsomorphin,11524144",ONC(=O)CCCCCN1C(=O)c2cccc3cccc(C1=O)c23;C1=CC2...,scriptaid;dorsomorphin;itdac-1;,5186;11524144;3785;
4,10.1371/journal.pone.0003531,Mus musculus,Primordial Germ Cells,Induced Pluripotent Epiblast Cells,10,"Trichostatin A,444732",not molecules,,


In [31]:
data_Drugs_metadata = pd.read_csv('DATA/Drugs_metadata.csv', index_col=0)
data_CD_signature_metadata = pd.read_csv('DATA/CD_signature_metadata.csv', index_col=0)

In [41]:
def number_sign_in_protocol(data_intersect_CFM_L1000FWD, conversion, what_counted):
    if what_counted == 'sign':
        n_col = 10
    elif what_counted == 'compound':
        n_col = 9
        
    data_intersect_CFM_L1000FWD_conv = data_intersect_CFM_L1000FWD[data_intersect_CFM_L1000FWD['conversion'] == conversion]
    
    number_what_counted = []
    list_what_counted = []
    for i in range(data_intersect_CFM_L1000FWD_conv.shape[0]):
        number_what_counted.append(len(set(data_intersect_CFM_L1000FWD_conv.iloc[i, n_col].split(';'))))
        list_what_counted = list_what_counted + list(set(data_intersect_CFM_L1000FWD_conv.iloc[i, n_col].split(';')))
    number_all_what_counted = len(set(list_what_counted))
    d = {'number_in_protocol': number_what_counted, 'number_all': number_all_what_counted}
    return d

In [56]:
def count_signatures_compounds(table_of_cell_conversion_and_chemicals, data_Drugs_metadata,
                               data_CD_signature_metadata):
    
    df_modernizated = modernization(
        table_of_cell_conversion_and_chemicals, data_Drugs_metadata, data_CD_signature_metadata)
    df_modernizated['conversion'] = \
        df_modernizated['Source Cell Type'] + '_' + \
        df_modernizated['Target Cell Type']
    
    list_conv = list(df_modernizated['conversion'].unique())
    list_number_protocol = []
    list_number_in_protocol_sign = []
    list_number_all_sign = []
    list_number_in_protocol_compound = []
    list_number_all_compound = []
    for conv in list_conv:
        list_number_protocol.append(df_modernizated[df_modernizated['conversion'] == conv].shape[0])
        d_sign  = number_sign_in_protocol(df_modernizated, conv, 'sign')
        list_number_in_protocol_sign.append(d_sign['number_in_protocol'])
        list_number_all_sign .append(d_sign['number_all'])
        
        d_compound  = number_sign_in_protocol(df_modernizated, conv, 'compound')
        list_number_in_protocol_compound.append(d_compound['number_in_protocol'])
        list_number_all_compound.append(d_compound['number_all'])
    df_count = pd.DataFrame(zip(list_conv, list_number_protocol, list_number_in_protocol_sign, list_number_all_sign,
                                list_number_in_protocol_compound,list_number_all_compound))
    df_count.columns = ['conversion','number of protocols',
                        'number of signatures in every protocol of the conversion', 
                        'number of all signatures for the conversion', 
                        'number of compounds in every protocol for the conversion', 
                        'number of all compounds for the conversion',]
    df_count = df_count.sort_values(by = 'number of protocols', ascending=False)
    return df_count

In [59]:
data_intersect_CFM_L1000FWD_1_counted = count_signatures_compounds(data_intersect_CFM_L1000FWD_1, 
                                                                data_Drugs_metadata, data_CD_signature_metadata)
data_intersect_CFM_L1000FWD_1_counted[data_intersect_CFM_L1000FWD_1_counted['number of protocols'] > 2].head(10)

,conversion,number of protocols,number of signatures in every protocol of the conversion,number of all signatures for the conversion,number of compounds in every protocol for the conversion,number of all compounds for the conversion
0,Fibroblasts_Induced Cardiomyocytes,19,"[40, 1, 75, 1, 1, 1011, 262, 2, 52, 7, 373, 41...",1574,"[2, 1, 5, 1, 1, 5, 9, 2, 4, 2, 13, 4, 2, 12, 1...",31
35,Fibroblasts_Induced Neurons,8,"[135, 131, 263, 160, 96, 86, 1, 218]",380,"[8, 8, 11, 9, 7, 6, 1, 10]",18
25,Fibroblasts_Induced Neural Stem Cells,6,"[30, 806, 29, 126, 126, 1]",977,"[4, 3, 5, 5, 5, 1]",12
1,Fibroblasts_Induced Pancreatic Beta Cells,6,"[1, 32, 184, 84, 91, 4]",388,"[1, 2, 10, 5, 8, 2]",22
3,Mesenchymal Stem Cells (Bone marrow derived)_I...,5,"[85, 75, 233, 75, 1]",317,"[3, 5, 15, 5, 1]",17
42,Fibroblasts_Induced Pluripotent Stem Cells,4,"[400, 274, 91, 29]",509,"[15, 8, 8, 3]",23
30,Fibroblasts_Induced Hepatocytes,3,"[1, 200, 160]",359,"[1, 9, 6]",14
23,Embryonic Stem Cells_Induced Cardiomyocytes,3,"[13, 4, 1]",16,"[2, 2, 1]",3


In [60]:
data_intersect_CFM_L1000FWD_07_counted = count_signatures_compounds(data_intersect_CFM_L1000FWD_07, 
                                                                data_Drugs_metadata, data_CD_signature_metadata)
data_intersect_CFM_L1000FWD_07_counted[data_intersect_CFM_L1000FWD_1_counted['number of protocols'] > 2].head(10)

,conversion,number of protocols,number of signatures in every protocol of the conversion,number of all signatures for the conversion,number of compounds in every protocol for the conversion,number of all compounds for the conversion
0,Fibroblasts_Induced Cardiomyocytes,19,"[40, 1, 86, 1, 1, 1041, 273, 2, 52, 7, 384, 41...",1636,"[3, 1, 6, 1, 1, 7, 11, 2, 5, 2, 14, 4, 5, 13, ...",39
35,Fibroblasts_Induced Neurons,8,"[146, 195, 314, 231, 107, 121, 1, 229]",526,"[10, 15, 13, 17, 8, 10, 1, 11]",31
25,Fibroblasts_Induced Neural Stem Cells,6,"[41, 806, 93, 137, 137, 12]",1041,"[5, 3, 11, 6, 6, 2]",18
1,Fibroblasts_Induced Pancreatic Beta Cells,6,"[1, 32, 248, 84, 164, 4]",525,"[1, 2, 16, 5, 19, 2]",39
3,Mesenchymal Stem Cells (Bone marrow derived)_I...,5,"[85, 75, 246, 75, 1]",330,"[4, 5, 19, 5, 2]",23
42,Fibroblasts_Induced Pluripotent Stem Cells,4,"[432, 285, 164, 29]",614,"[17, 9, 19, 3]",36
30,Fibroblasts_Induced Hepatocytes,3,"[12, 211, 171]",370,"[2, 10, 7]",15
23,Embryonic Stem Cells_Induced Cardiomyocytes,3,"[24, 4, 1]",27,"[3, 2, 1]",4
